In [1]:
import os
if os.name != 'posix':
    %matplotlib ipympl
else:
    %matplotlib notebook
import scipy.constants as sc
from scipy.optimize import fsolve

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
})
import os
import h5py

print(os.getpid())
%cd ../../

import pylib.mix as mix
import pylib.measurement as mse

6557
/media/work/docs/codes/QuCF/scripts-py


In [2]:
# --- Launch to change cells' width ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

colors_ = ["blue", "red", "green", "gray", "black"]

from matplotlib import ticker
cmap_two_signs_= 'coolwarm'
cmap_two_signs_= 'Blues'
# cmap_two_signs_= 'Greys'
# cmap_two_signs_= 'Pastel1'
# cmap_two_signs_= 'flag'
# cmap_two_signs_= 'nipy_spectral'
# cmap_two_signs_= 'jet'
# cmap_two_signs_= 'seismic'
def fmt_(x, pos):
    a, b = '{:.2e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

In [3]:
for i in range(10):
    plt.close()

In [38]:
# ---------------------------------------------------------
# --- Compute angles for the compression gadget ---
# ---------------------------------------------------------
mix.reload_module(mix)

N_mult = 200
nc = mix.compute_nc_for_compression_gadget(N_mult)
ay_one = 0.64

flag_print_all = False

print()
print("nc \t\t{:d}".format(nc))
print("N_mult \t\t{:d}".format(N_mult))
print("ay_one \t{:0.12e}".format(ay_one))

# # --- Results ---
# print("\nReference results")
# c1 = np.cos(ay_one/2.)
# ress = np.zeros(N_mult)
# ress[0] = c1
# for ii in range(1, N_mult):
#     ress[ii] = ress[ii-1] * c1
    
# if flag_print_all:
#     for ii in range(N_mult):    
#         print("i = {:d}, res = {:0.3e}".format(ii+1, ress[ii]))
    
# --- Compute angles for the Rc-gadget:
ay = np.linspace(np.pi/6., np.pi/3., N_mult)
az = np.linspace(np.pi/8., np.pi/4., N_mult)

filename_angles = "../QuCF/tests/compression-gadget/RC_TEST.hdf5"
print("\n\nWrite angles to:\n " + filename_angles)
with h5py.File(filename_angles, "w") as f:
    grp = f.create_group("angles")   
    grp.create_dataset('ay', data=ay)
    grp.create_dataset('az', data=az)
del filename_angles
    
    
# --- Results from Rc-gadget ---
print("\nReference results from Rc-gadget")
ress = np.zeros(N_mult, dtype=complex)
ress[0] = np.cos(ay[0]/2.) * np.exp(-1j*az[0]/2.)
for ii in range(1, N_mult):
    ress[ii] = ress[ii-1] * np.cos(ay[ii]/2.) * np.exp(-1j*az[ii]/2.)
    
if flag_print_all:
    for ii in range(N_mult):    
        print("i = {:d}, res = {:20.3e}".format(ii+1, ress[ii]))


nc 		9
N_mult 		200
ay_one 	6.400000000000e-01


Write angles to:
 ../QuCF/tests/compression-gadget/RC_TEST.hdf5

Reference results from Rc-gadget


In [39]:
# ----------------------------------------------------------
# --- Read results from the QC simulations ---
# ----------------------------------------------------------
mix.reload_module(mse)


om = mse.MeasOracle__(True)

om.path_ = "../QuCF/tests/compression-gadget/"
om.pname_ = "compression"
om.open()

dd_gadget = om.read_gadget("G1", 0)

if flag_print_all:
    for ii in range(dd_gadget["N-mult"]):
        print("i = {:d}, QC-res, CL-res =   {:20.3e},   {:20.3e}".format(ii+1, dd_gadget["ampls"][ii][0], ress[ii]))

print()
max_abs_err = np.abs(dd_gadget["ampls"][ii][0] - ress[ii])
for ii in range(dd_gadget["N-mult"]):
    temp = np.abs(dd_gadget["ampls"][ii][0] - ress[ii])
    if temp > max_abs_err:
        max_abs_err = temp
    if flag_print_all:
        print("i = {:d}, abs-diff = {:0.3e}".format(ii+1, temp))
del temp
    
print("maximum absolute error: {:0.3e}".format(max_abs_err))

Reading the file ../QuCF/tests/compression-gadget//compression_OUTPUT.hdf5...
Name of the simulation is compression
Simulation has been performed  11-27-2023 10:09:34

maximum absolute error: 2.114e-15


In [87]:
dd_gadget["ampls"][0]

array([0.949-5.812e-17j])

In [81]:
dd_gadget["states"][0]

array([[0, 0, 0, 1, 1, 1, 0],
       [1, 0, 1, 0, 0, 0, 0]], dtype=int16)

In [83]:
om.dd_["reg-shifts"]

{'ae': 0, 'rc': 1, 'j': 6}

In [ ]:
# # ---------------------------------------------------------
# # --- Compute angles for the compression gadget ---
# # ---------------------------------------------------------
# mix.reload_module(mix)

# a_max  = 0.2
# N_mult = 4
# nc = mix.compute_nc_for_compression_gadget(N_mult)

# ay_one = 0.64

# alpha_s_0 = - a_max
# alpha_s_1 = a_max * N_mult/(N_mult - 1)
# print()
# print("nc \t\t{:d}".format(nc))
# print("N_mult \t\t{:d}".format(N_mult))
# print("alpha_s_0 \t{:0.12e}".format(alpha_s_0))
# print("alpha_s_1 \t{:0.12e}".format(alpha_s_1))

# # --- compute angles for the Rc-gadget:
# ay = np.linspace(np.pi/6., np.pi/3., N_mult)
# az = np.linspace(np.pi/8., np.pi/4., N_mult)

# filename_angles = "../QuCF/tests/compression-gadget/RC_TEST.hdf5"
# print("write angles to:\n " + filename_angles)
# with h5py.File(filename_angles, "w") as f:
#     grp = f.create_group("angles")   
#     grp.create_dataset('ay', data=ay)
#     grp.create_dataset('az', data=az)
# del ay, az, filename_angles

# # ---
# d_alpha_s = dx = (2. * alpha_s_1) / N_mult